In [61]:
import pandas as pd

### Дано:

In [88]:
x_options = [0, 100, 200, 300, 400, 500, 600, 700, 800, 900]

def P(x):
    vals = [0, 150, 280, 410, 540, 660, 780, 890, 1000, 1100]
    if int(x / 100) >= len(vals):
        return -1
    return vals[int(x / 100)]
def Q(x):
    vals = [0, 10, 20, 30, 50, 70, 100]
    if int(x / 100) >= len(vals):
        return -1
    return vals[int(x / 100)]

N = 3
delta = 100
E = 300
alpha = 300

x = [x for x in range(N + 1)]
u = [x + 1 for x in range(N)]

print(x)
print(u)

[0, 1, 2, 3]
[1, 2, 3]


In [63]:
def change_law(k):
    x_k_next = x[k] + u[k]
    return x_k_next

def spendings(k):
    d = P(u[k]) + Q(x[k])
    return d

### Шаги

$$S_1(x_2) = \min\lbrace P(u_3) + Q(x_2) + S_0(x_2 + u_3 - \alpha): u_3 = \max\lbrace \alpha - x_2; 0\rbrace \leq u_3 \leq \min\lbrace 1\alpha - x_2; \alpha + E \rbrace \rbrace$$

$$S_2(x_1) = \min\lbrace P(u_2) + Q(x_1) + S_1(x_1 + u_2 - \alpha): u_2 = \max\lbrace \alpha - x_1; 0\rbrace  \leq u_2 \leq \min\lbrace 2\alpha - x_1; \alpha + E \rbrace \rbrace$$

$$S_3(x_0) = \min\lbrace P(u_1) + Q(x_0) + S_2(x_0 + u_1 - \alpha): u_1 = \max\lbrace \alpha - x_0; 0\rbrace \leq u_1 \leq \min\lbrace 3\alpha - x_0; \alpha + E \rbrace \rbrace$$

In [91]:
def S(k, x):
    if k == 0:
        return (0, 0)
    
    def get_lower(x):
        return max(alpha - x, 0)
    def get_upper(x):
        return min(k * alpha - x, alpha + E)
    def get_possible_u(x):
        possible_u = []
        t = get_lower(x)
        while t <= get_upper(x):
            possible_u.append(t)
            t += delta
        return possible_u
    possible_S = []
    possible_u = get_possible_u(x)
    
    if len(possible_u) == 0:
        return (-1, 0)

    for u in possible_u:
        if Q(x) == -1 or P(x) == -1:
            return (-1, 0)
        s = P(u) + Q(x) + S(k-1, x + u - alpha)[0]
        possible_S.append(s)
    u = []
    min_S = min(possible_S)
    for s in enumerate(possible_S):
        if s[1] == min_S:
            u.append(possible_u[s[0]])
    return (min_S, u)

In [92]:
def step(k):
    print("k =", k)
    print("k + 1 = ", k+1)
    print("x idx = ", N - k)
    print("u idx = ", N - k + 1)
    print("S prev idx = ", k - 1)
    print()

    s_str = " + S{}(x{} + u{} - alpha)".format(k - 1, N - k, N - k + 1) if k - 1 > 0 else ""
    print("min{{P(u{}) + Q(x{}){}}}".format(N - k + 1, N - k, s_str))
    print()

    c1 = "x{}".format(N - k)
    c2 = "S{}(x{})".format(k, N - k)
    c3 = "u*{}({})".format(N - k + 1, N - k)

    df = pd.DataFrame(columns=[c1, c2, c3])
    for x in [0, 100, 200, 300, 400, 500, 600, 700]:
        min_S, u = S(k, x)
        if min_S == -1:
            break
        row = [x, min_S, u]
        df.loc[len(df)] = row

    print(df)

In [93]:
step(3)

k = 3
k + 1 =  4
x idx =  0
u idx =  1
S prev idx =  2

min{P(u1) + Q(x0) + S2(x0 + u1 - alpha)}

    x0  S3(x0)      u*1(0)
0    0    1220  [300, 600]
1  100    1100       [200]
2  200     980       [100]
3  300     840         [0]
4  400     750         [0]
5  500     650         [0]
6  600     540         [0]


In [94]:
step(2)

k = 2
k + 1 =  3
x idx =  1
u idx =  2
S prev idx =  1

min{P(u2) + Q(x1) + S1(x1 + u2 - alpha)}

    x1  S2(x1)      u*2(1)
0    0     810       [600]
1  100     700  [200, 500]
2  200     580       [100]
3  300     440         [0]
4  400     340         [0]
5  500     240         [0]
6  600     130         [0]


In [95]:
step(1)

k = 1
k + 1 =  2
x idx =  2
u idx =  3
S prev idx =  0

min{P(u3) + Q(x2)}

    x2  S1(x2) u*3(2)
0    0     410  [300]
1  100     290  [200]
2  200     170  [100]
3  300      30    [0]
